# JBang Jupyter Kernel Examples

JJava supports using jbang for loading dependencies/resources.

In [1]:
%jbang properties@jbangdev

Create a DataFrame and plot it with ECharts.

In [ ]:
%jbang MyClass.java
new mypackage.MyClass("Max", 30).getName();

In [11]:
%%jbang
//DEPS com.github.javafaker:javafaker:1.0.2
import com.github.javafaker.Faker;

Faker faker = new Faker();




In [ ]:
import com.github.javafaker.Faker;

Faker faker = new Faker();

faker.name().fullName();

In [1]:
System.getenv();


{VSCODE_CODE_CACHE_PATH=/Users/max/Library/Application Support/Cursor/CachedData/adb0f9e3e4f184bba7f3fa6dbfd72ad0ebb8cfd0, SDKMAN_CANDIDATES_API=https://api.sdkman.io/2, ORIGINAL_XDG_CURRENT_DESKTOP=undefined, TERM_FEATURES=T3LrMSc7UUw9Ts3BFGsSyHNoSxF, TERM_SESSION_ID=w0t0p0:7A5F8165-89E2-496A-8BD7-D126647F8156, PYTHONUNBUFFERED=1, VSCODE_CRASH_REPORTER_PROCESS_TYPE=extensionHost, __MISE_ORIG_PATH=/opt/homebrew/opt/mise/bin:/Users/max/.sdkman/candidates/maven/current/bin:/Users/max/.sdkman/candidates/java/current/bin:/Users/max/.local/bin:/Users/max/.jbang/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/opt/homebrew/bin:/Applications/iTerm.app/Contents/Resources/utilities, VSCODE_ESM_ENTRYPOINT=vs/workbench/api/node/extensionHostProce

In [4]:
var id = display("Markdown *test* **snippet**:\n* bullet 1\n* bullet 2", "text/markdown")

Markdown *test* **snippet**:
* bullet 1
* bullet 2